<a href="https://colab.research.google.com/github/du6293/DL_study/blob/main/6%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 학습 관련 기술들


## 1. 확률적 경사 하강법(SGD)

In [ ]:
class SGD:
  def __init__(self, lr = 0.01)
    self.lr = lr

  def update(self, params, grads):
    for key in params.keys():
      params[key] -= self.lr * grads[key]

## 2. 모멘텀

In [ ]:
class Momentum:
  def __init__(self, lr = 0.01, momentum = 0.9)
    self.lr = lr
    self.momentum = momentum
    self.v = None  # 속도
  def update(self, params, grads):
    if self.v is None:
      self.v = {}
      for key , val in params.items():
        self.v[key] = np.zeros_like(val)
      
    for key in params.keys():
      self.v[key] = self.momentum * self.v[key] - self.lr * grad[key]
      params[key] += self.v[key]

## 3. AdaGrad

In [ ]:
class AdaGrad:
  def __init__(self, lr= 0.01):
    self.lr = lr
    self.h = None
  
  def update(self, params, grads):
    if self.h is None:
      self.h = {}
      
      for key, val in params.items():
        self.h[key] = np.zeros_like(val)
    
    for key in params.keys():
      self.h[key] += grads[key] * grads[key]
      prarams[key] -= self.lr * grads[key] / (np.sqrt(self.h[key])) + le-7  # self.h[key]에 0이 담겨져 있다고 해도 0으로 나누는 사태를 막음

## 4. RMSProp

In [ ]:
class RMSProp:
  def __init__ (self, lr = 0.01, decay_rate = 0.99 )
    self.lr = lr
    self.decay_rate = decay_rate
    self.h = None
  
  def update(self, params, grads):
    if self.h is None:
      self.h = {}
      
      for key, val in params,items():
        self.h[key] = np.zeros_like(val)

    for key in params.keys():
      self.h[key] *= self.decay_rate
      self.h[key] += (1 - self.decay_rate) * grads[key] * grads[key]
      params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)



## 4. Adam

In [ ]:
class Adam:  # 모멘텀 + AdaGrad
  def __init__(self, lr = 0.01, beta = 0.9, beta2 = 0.999 ):
    self.lr = lr
    self.beta1 = beta1
    self.beta2 = beta2
    self.iter = 0
    self.m = None
    self.v = None

  def update(self, params, grads):
    if self.m is None:
      self.m, self.v = {},{}
      for key, val in params.items():
        self.m[key] = np.zeros_like(val)
        self.v[key] = np.zeros_like(val)

    self.iter += 1
    lr_t = self.lr * np.sqrt(1.0 - self.beta2 ** self.iter) / (1.0 - self.beta1 ** self.iter)


    for key in params.keys():
      self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
      self.v[key] += (1 - self.beta2) * (grads[key] ** 2 - self.v[key])

      params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e - 7) 


### MNIST 데이터셋으로 본 갱신 방법 비교

In [ ]:
# coding: utf-8
import os
import sys
import numpy as np
sys.path.append("/content/drive/MyDrive/bottom/deep-learning-from-scratch-master/deep-learning-from-scratch-master/ch06")  # 부모 디렉터리의 파일을 가져올 수 있도록 설정

import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.util import smooth_curve
from common.multi_layer_net import MultiLayerNet
from common.optimizer import *


# 0. MNIST 데이터 읽기==========
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

train_size = x_train.shape[0]
batch_size = 128
max_iterations = 2000


# 1. 실험용 설정==========
optimizers = {}
optimizers['SGD'] = SGD()
optimizers['Momentum'] = Momentum()
optimizers['AdaGrad'] = AdaGrad()
optimizers['Adam'] = Adam()
#optimizers['RMSprop'] = RMSprop()

networks = {}
train_loss = {}
for key in optimizers.keys():
    networks[key] = MultiLayerNet(
        input_size=784, hidden_size_list=[100, 100, 100, 100],
        output_size=10)
    train_loss[key] = []    


# 2. 훈련 시작==========
for i in range(max_iterations):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    for key in optimizers.keys():
        grads = networks[key].gradient(x_batch, t_batch)
        optimizers[key].update(networks[key].params, grads)
    
        loss = networks[key].loss(x_batch, t_batch)
        train_loss[key].append(loss)
    
    if i % 100 == 0:
        print( "===========" + "iteration:" + str(i) + "===========")
        for key in optimizers.keys():
            loss = networks[key].loss(x_batch, t_batch)
            print(key + ":" + str(loss))


# 3. 그래프 그리기==========
markers = {"SGD": "o", "Momentum": "x", "AdaGrad": "s", "Adam": "D"}
x = np.arange(max_iterations)
for key in optimizers.keys():
    plt.plot(x, smooth_curve(train_loss[key]), marker=markers[key], markevery=100, label=key)
plt.xlabel("iterations")
plt.ylabel("loss")
plt.ylim(0, 1)
plt.legend()
plt.show()


#### **은닉층의 활성화값 분포**

가중치의 초깃값에 따른 은닉층 활성화값(활성화 함수의 출력 데이터)들의 변화

5층 신경망에 무작위로 생성한 입력 데이터를 흘리며 각 층의 활성화값 분포를 히스토그램으로 그리기

## 1.표준편차를 1로

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def ReLU(x):
  return np.maximum(0,x)
  
def tanh(x):
  return np.tang(x)


input_data = np.random.randn(1000,100)  # 1000개의 데이터 >> 정규분포로 무작위로 생성헤 5층 신경망에 흘림
node_num = 100 # 각 은닉층의 노드(뉴런) 수
hidden_layer_size = 5  # 은닉층이 5개
activations = {}  # 이곳에 각 층의 활성화 결과를 저장

x = input_data

for i in range(hidden_layer_size):
  if i != 0:
    x = activations[i-1]

  # 초깃값을 다양하게 바꿔가며 실험해보자!
  w = np.random.randn(node_num, node_num) * 1
  # w = np.random.randn(node_num, node_num) * np.sqrt(1.0 / node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(2.0 / node_num)
  a = np.dot(x,w)


  #활성화 함수도 바꿔가며 실험해보자
  z = sigmoid(a)
  # z = ReLU(a)
  # z = tanh(a)


  activations[i] = z

In [ ]:
# 히스토그램 그리기
for i , a in activations.items():
  plt.subplot(1,len(activations), i+1)
  plt.title(str(i+1) + "layer")
  if i != 0 :
    plt.yticks([],[])
    plt.hist(a.flatten(), 30, range=(0,1))
plt.show()

기울기 소실 >> 딥러닝에서 심각한 문제가 된다.
각 층의 활성화 값들이 0과 1에 치우쳐 분포
시그모이드 함수는 출력이 0과 1에 가까워지자 미분이 0에 다가간다.
따라서 기울기값이 0과 1에 치우쳐 분포하게 되면 역전파의 기울기 값이 점점 작아지다가 사라진다.

## 2.표준편차를 0.01로

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def ReLU(x):
  return np.maximum(0,x)
  
def tanh(x):
  return np.tang(x)


input_data = np.random.randn(1000,100)  # 1000개의 데이터 >> 정규분포로 무작위로 생성헤 5층 신경망에 흘림
node_num = 100 # 각 은닉층의 노드(뉴런) 수
hidden_layer_size = 5  # 은닉층이 5개
activations = {}  # 이곳에 각 층의 활성화 결과를 저장

x = input_data

for i in range(hidden_layer_size):
  if i != 0:
    x = activations[i-1]

  # 초깃값을 다양하게 바꿔가며 실험해보자!
  w = np.random.randn(node_num, node_num) * 0.01
  # w = np.random.randn(node_num, node_num) * np.sqrt(1.0 / node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(2.0 / node_num)
  a = np.dot(x,w)


  #활성화 함수도 바꿔가며 실험해보자
  z = sigmoid(a)
  # z = ReLU(a)
  # z = tanh(a)


  activations[i] = z

In [ ]:
# 히스토그램 그리기
for i , a in activations.items():
  plt.subplot(1,len(activations), i+1)
  plt.title(str(i+1) + "layer")
  if i != 0 :
    plt.yticks([],[])
    plt.hist(a.flatten(), 30, range=(0,1))
plt.show()

표현력 제한
0.5부근에 활성화값들이 집중됨 >> 다수의 뉴런이 거의 같은 값을 출력하고 있으니 뉴런을 여러 개 둔 의미가 없어진다.

### Xavier초깃값 - sigmoid함수

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def ReLU(x):
  return np.maximum(0,x)
  
def tanh(x):
  return np.tang(x)


input_data = np.random.randn(1000,100)  # 1000개의 데이터 >> 정규분포로 무작위로 생성헤 5층 신경망에 흘림
node_num = 100 # 각 은닉층의 노드(뉴런) 수
hidden_layer_size = 5  # 은닉층이 5개
activations = {}  # 이곳에 각 층의 활성화 결과를 저장

x = input_data

for i in range(hidden_layer_size):
  if i != 0:
    x = activations[i-1]

  # 초깃값을 다양하게 바꿔가며 실험해보자!
  w = np.random.randn(node_num, node_num) / np.sqrt(node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(1.0 / node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(2.0 / node_num)
  a = np.dot(x,w)


  #활성화 함수도 바꿔가며 실험해보자
  z = sigmoid(a)
  # z = ReLU(a)
  # z = tanh(a)


  activations[i] = z

In [ ]:
# 히스토그램 그리기
for i , a in activations.items():
  plt.subplot(1,len(activations), i+1)
  plt.title(str(i+1) + "layer")
  if i != 0 :
    plt.yticks([],[])
    plt.hist(a.flatten(), 30, range=(0,1))
plt.show()

### Xavier초깃값 - tanh함수

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def ReLU(x):
  return np.maximum(0,x)
  
def tanh(x):
  return np.tanh(x)


input_data = np.random.randn(1000,100)  # 1000개의 데이터 >> 정규분포로 무작위로 생성헤 5층 신경망에 흘림
node_num = 100 # 각 은닉층의 노드(뉴런) 수
hidden_layer_size = 5  # 은닉층이 5개
activations = {}  # 이곳에 각 층의 활성화 결과를 저장

x = input_data

for i in range(hidden_layer_size):
  if i != 0:
    x = activations[i-1]

  # 초깃값을 다양하게 바꿔가며 실험해보자!
  w = np.random.randn(node_num, node_num) / np.sqrt(node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(1.0 / node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(2.0 / node_num)
  a = np.dot(x,w)


  #활성화 함수도 바꿔가며 실험해보자
  # z = sigmoid(a)
  # z = ReLU(a)
  z = tanh(a)


  activations[i] = z

In [ ]:
# 히스토그램 그리기
for i , a in activations.items():
  plt.subplot(1,len(activations), i+1)
  plt.title(str(i+1) + "layer")
  if i != 0 :
    plt.yticks([],[])
    plt.hist(a.flatten(), 30, range=(0,1))
plt.show()

# Xavier초깃값 - ReLU함수

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def ReLU(x):
  return np.maximum(0,x)
  
def tanh(x):
  return np.tanh(x)


input_data = np.random.randn(1000,100)  # 1000개의 데이터 >> 정규분포로 무작위로 생성헤 5층 신경망에 흘림
node_num = 100 # 각 은닉층의 노드(뉴런) 수
hidden_layer_size = 5  # 은닉층이 5개
activations = {}  # 이곳에 각 층의 활성화 결과를 저장

x = input_data

for i in range(hidden_layer_size):
  if i != 0:
    x = activations[i-1]

  # 초깃값을 다양하게 바꿔가며 실험해보자!
  w = np.random.randn(node_num, node_num) / np.sqrt(node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(1.0 / node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(2.0 / node_num)
  a = np.dot(x,w)


  #활성화 함수도 바꿔가며 실험해보자
  # z = sigmoid(a)
  z = ReLU(a)
  # z = tanh(a)


  activations[i] = z

In [ ]:
# 히스토그램 그리기
for i , a in activations.items():
  plt.subplot(1,len(activations), i+1)
  plt.title(str(i+1) + "layer")
  if i != 0 :
    plt.yticks([],[])
    plt.hist(a.flatten(), 30, range=(0,1))
plt.show()

# std = 0.01인 정규분포 & ReLU함수 사용

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def ReLU(x):
  return np.maximum(0,x)
  
def tanh(x):
  return np.tanh(x)


input_data = np.random.randn(1000,100)  # 1000개의 데이터 >> 정규분포로 무작위로 생성헤 5층 신경망에 흘림
node_num = 100 # 각 은닉층의 노드(뉴런) 수
hidden_layer_size = 5  # 은닉층이 5개
activations = {}  # 이곳에 각 층의 활성화 결과를 저장

x = input_data

for i in range(hidden_layer_size):
  if i != 0:
    x = activations[i-1]

  # 초깃값을 다양하게 바꿔가며 실험해보자!
  w = np.random.randn(node_num, node_num) * 0.01
  # w = np.random.randn(node_num, node_num) * np.sqrt(1.0 / node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(2.0 / node_num)
  a = np.dot(x,w)


  #활성화 함수도 바꿔가며 실험해보자
  # z = sigmoid(a)
  z = ReLU(a)
  # z = tanh(a)


  activations[i] = z

In [ ]:
# 히스토그램 그리기
for i , a in activations.items():
  plt.subplot(1,len(activations), i+1)
  plt.title(str(i+1) + "layer")
  if i != 0 :
    plt.yticks([],[])
    plt.hist(a.flatten(), 30, range=(0,1))
plt.show()

# Xavier초깃값 & ReLU함수

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def ReLU(x):
  return np.maximum(0,x)
  
def tanh(x):
  return np.tanh(x)


input_data = np.random.randn(1000,100)  # 1000개의 데이터 >> 정규분포로 무작위로 생성헤 5층 신경망에 흘림
node_num = 100 # 각 은닉층의 노드(뉴런) 수
hidden_layer_size = 5  # 은닉층이 5개
activations = {}  # 이곳에 각 층의 활성화 결과를 저장

x = input_data

for i in range(hidden_layer_size):
  if i != 0:
    x = activations[i-1]

  # 초깃값을 다양하게 바꿔가며 실험해보자!
  w = np.random.randn(node_num, node_num) / np.sqrt(node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(1.0 / node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(2.0 / node_num)
  a = np.dot(x,w)


  #활성화 함수도 바꿔가며 실험해보자
  # z = sigmoid(a)
  z = ReLU(a)
  # z = tanh(a)


  activations[i] = z

In [ ]:
# 히스토그램 그리기
for i , a in activations.items():
  plt.subplot(1,len(activations), i+1)
  plt.title(str(i+1) + "layer")
  if i != 0 :
    plt.yticks([],[])
    plt.hist(a.flatten(), 30, range=(0,1))
plt.show()

# He 초깃값 & ReLU함수

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def ReLU(x):
  return np.maximum(0,x)
  
def tanh(x):
  return np.tanh(x)


input_data = np.random.randn(1000,100)  # 1000개의 데이터 >> 정규분포로 무작위로 생성헤 5층 신경망에 흘림
node_num = 100 # 각 은닉층의 노드(뉴런) 수
hidden_layer_size = 5  # 은닉층이 5개
activations = {}  # 이곳에 각 층의 활성화 결과를 저장

x = input_data

for i in range(hidden_layer_size):
  if i != 0:
    x = activations[i-1]

  # 초깃값을 다양하게 바꿔가며 실험해보자!
  # w = np.random.randn(node_num, node_num) / np.sqrt(node_num)
  # w = np.random.randn(node_num, node_num) * np.sqrt(1.0 / node_num)
  w = np.random.randn(node_num, node_num) * np.sqrt(2.0 / node_num)
  a = np.dot(x,w)


  #활성화 함수도 바꿔가며 실험해보자
  # z = sigmoid(a)
  z = ReLU(a)
  # z = tanh(a)


  activations[i] = z

In [ ]:
# 히스토그램 그리기
for i , a in activations.items():
  plt.subplot(1,len(activations), i+1)
  plt.title(str(i+1) + "layer")
  if i != 0 :
    plt.yticks([],[])
    plt.hist(a.flatten(), 30, range=(0,1))
plt.show()

### MNIST 데이터셋으로 본 가중치 초깃값 비교

In [ ]:
# coding: utf-8
import os
import sys

sys.path.append("/content/drive/MyDrive/bottom/deep-learning-from-scratch-master/deep-learning-from-scratch-master/ch06")  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.util import smooth_curve
from common.multi_layer_net import MultiLayerNet
from common.optimizer import SGD


# 0. MNIST 데이터 읽기==========
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

train_size = x_train.shape[0]
batch_size = 128
max_iterations = 2000


# 1. 실험용 설정==========
weight_init_types = {'std=0.01': 0.01, 'Xavier': 'sigmoid', 'He': 'relu'}
optimizer = SGD(lr=0.01)

networks = {}
train_loss = {}
for key, weight_type in weight_init_types.items():
    networks[key] = MultiLayerNet(input_size=784, hidden_size_list=[100, 100, 100, 100],
                                  output_size=10, weight_init_std=weight_type)
    train_loss[key] = []


# 2. 훈련 시작==========
for i in range(max_iterations):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    for key in weight_init_types.keys():
        grads = networks[key].gradient(x_batch, t_batch)
        optimizer.update(networks[key].params, grads)
    
        loss = networks[key].loss(x_batch, t_batch)
        train_loss[key].append(loss)
    
    if i % 100 == 0:
        print("===========" + "iteration:" + str(i) + "===========")
        for key in weight_init_types.keys():
            loss = networks[key].loss(x_batch, t_batch)
            print(key + ":" + str(loss))


# 3. 그래프 그리기==========
markers = {'std=0.01': 'o', 'Xavier': 's', 'He': 'D'}
x = np.arange(max_iterations)
for key in weight_init_types.keys():
    plt.plot(x, smooth_curve(train_loss[key]), marker=markers[key], markevery=100, label=key)
plt.xlabel("iterations")
plt.ylabel("loss")
plt.ylim(0, 2.5)
plt.legend()
plt.show()


###  배치 정규화의 효과

In [ ]:
# coding: utf-8
import sys, os
sys.path.append("/content/drive/MyDrive/bottom/deep-learning-from-scratch-master/deep-learning-from-scratch-master/ch03")  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.multi_layer_net_extend import MultiLayerNetExtend
from common.optimizer import SGD, Adam

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

# 학습 데이터를 줄임
x_train = x_train[:1000]
t_train = t_train[:1000]

max_epochs = 20
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.01


def __train(weight_init_std):
    bn_network = MultiLayerNetExtend(input_size=784, hidden_size_list=[100, 100, 100, 100, 100], output_size=10, 
                                    weight_init_std=weight_init_std, use_batchnorm=True)
    network = MultiLayerNetExtend(input_size=784, hidden_size_list=[100, 100, 100, 100, 100], output_size=10,
                                weight_init_std=weight_init_std)
    optimizer = SGD(lr=learning_rate)
    
    train_acc_list = []
    bn_train_acc_list = []
    
    iter_per_epoch = max(train_size / batch_size, 1)
    epoch_cnt = 0
    
    for i in range(1000000000):
        batch_mask = np.random.choice(train_size, batch_size)
        x_batch = x_train[batch_mask]
        t_batch = t_train[batch_mask]
    
        for _network in (bn_network, network):
            grads = _network.gradient(x_batch, t_batch)
            optimizer.update(_network.params, grads)
    
        if i % iter_per_epoch == 0:
            train_acc = network.accuracy(x_train, t_train)
            bn_train_acc = bn_network.accuracy(x_train, t_train)
            train_acc_list.append(train_acc)
            bn_train_acc_list.append(bn_train_acc)
    
            print("epoch:" + str(epoch_cnt) + " | " + str(train_acc) + " - " + str(bn_train_acc))
    
            epoch_cnt += 1
            if epoch_cnt >= max_epochs:
                break
                
    return train_acc_list, bn_train_acc_list


# 그래프 그리기==========
weight_scale_list = np.logspace(0, -4, num=16)
x = np.arange(max_epochs)

for i, w in enumerate(weight_scale_list):
    print( "============== " + str(i+1) + "/16" + " ==============")
    train_acc_list, bn_train_acc_list = __train(w)
    
    plt.subplot(4,4,i+1)
    plt.title("W:" + str(w))
    if i == 15:
        plt.plot(x, bn_train_acc_list, label='Batch Normalization', markevery=2)
        plt.plot(x, train_acc_list, linestyle = "--", label='Normal(without BatchNorm)', markevery=2)
    else:
        plt.plot(x, bn_train_acc_list, markevery=2)
        plt.plot(x, train_acc_list, linestyle="--", markevery=2)

    plt.ylim(0, 1.0)
    if i % 4:
        plt.yticks([])
    else:
        plt.ylabel("accuracy")
    if i < 12:
        plt.xticks([])
    else:
        plt.xlabel("epochs")
    plt.legend(loc='lower right')
    
plt.show()


### 오버피팅 일으키기

In [ ]:
# coding: utf-8
import os
import sys

sys.path.append("/content/drive/MyDrive/bottom/deep-learning-from-scratch-master/deep-learning-from-scratch-master/ch03")  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from common.multi_layer_net import MultiLayerNet
from common.optimizer import SGD

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

# 오버피팅을 재현하기 위해 학습 데이터 수를 줄임
x_train = x_train[:300]
t_train = t_train[:300]

# weight decay（가중치 감쇠） 설정 =======================
#weight_decay_lambda = 0 # weight decay를 사용하지 않을 경우
weight_decay_lambda = 0.1
# ====================================================

network = MultiLayerNet(input_size=784, hidden_size_list=[100, 100, 100, 100, 100, 100], output_size=10,
                        weight_decay_lambda=weight_decay_lambda)
optimizer = SGD(lr=0.01) # 학습률이 0.01인 SGD로 매개변수 갱신

max_epochs = 201
train_size = x_train.shape[0]
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
epoch_cnt = 0

for i in range(1000000000):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grads = network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print("epoch:" + str(epoch_cnt) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc))

        epoch_cnt += 1
        if epoch_cnt >= max_epochs:
            break


# 그래프 그리기==========
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, train_acc_list, marker='o', label='train', markevery=10)
plt.plot(x, test_acc_list, marker='s', label='test', markevery=10)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()


### 가중치 감소

In [ ]:
# # coding: utf-8
# import os
# import sys

# sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
# import numpy as np
# import matplotlib.pyplot as plt
# from dataset.mnist import load_mnist
# from common.multi_layer_net import MultiLayerNet
# from common.optimizer import SGD


# coding: utf-8
import sys, os
sys.path.append("/content/drive/MyDrive/bottom/deep-learning-from-scratch-master/deep-learning-from-scratch-master/ch03")  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class MultiLayerNet:
    """완전연결 다층 신경망

    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    weight_decay_lambda : 가중치 감소(L2 법칙)의 세기
    """
    def __init__(self, input_size, hidden_size_list, output_size,
                 activation='relu', weight_init_std='relu', weight_decay_lambda=0):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size_list = hidden_size_list
        self.hidden_layer_num = len(hidden_size_list)
        self.weight_decay_lambda = weight_decay_lambda
        self.params = {}

        # 가중치 초기화
        self.__init_weight(weight_init_std)

        # 계층 생성
        activation_layer = {'sigmoid': Sigmoid, 'relu': Relu}
        self.layers = OrderedDict()
        for idx in range(1, self.hidden_layer_num+1):
            self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)],
                                                      self.params['b' + str(idx)])
            self.layers['Activation_function' + str(idx)] = activation_layer[activation]()

        idx = self.hidden_layer_num + 1
        self.layers['Affine' + str(idx)] = Affine(self.params['W' + str(idx)],
            self.params['b' + str(idx)])

        self.last_layer = SoftmaxWithLoss()

    def __init_weight(self, weight_init_std):
        """가중치 초기화
        
        Parameters
        ----------
        weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
            'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
            'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
        """
        all_size_list = [self.input_size] + self.hidden_size_list + [self.output_size]
        for idx in range(1, len(all_size_list)):
            scale = weight_init_std
            if str(weight_init_std).lower() in ('relu', 'he'):
                scale = np.sqrt(2.0 / all_size_list[idx - 1])  # ReLU를 사용할 때의 권장 초깃값
            elif str(weight_init_std).lower() in ('sigmoid', 'xavier'):
                scale = np.sqrt(1.0 / all_size_list[idx - 1])  # sigmoid를 사용할 때의 권장 초깃값
            self.params['W' + str(idx)] = scale * np.random.randn(all_size_list[idx-1], all_size_list[idx])
            self.params['b' + str(idx)] = np.zeros(all_size_list[idx])

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """손실 함수를 구한다.
        
        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블 
        
        Returns
        -------
        손실 함수의 값
        """
        y = self.predict(x)

        weight_decay = 0
        for idx in range(1, self.hidden_layer_num + 2):
            W = self.params['W' + str(idx)]
            weight_decay += 0.5 * self.weight_decay_lambda * np.sum(W ** 2)

        return self.last_layer.forward(y, t) + weight_decay

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        """기울기를 구한다(수치 미분).
        
        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        
        Returns
        -------
        각 층의 기울기를 담은 딕셔너리(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        for idx in range(1, self.hidden_layer_num+2):
            grads['W' + str(idx)] = numerical_gradient(loss_W, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_W, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        
        Returns
        -------
        각 층의 기울기를 담은 딕셔너리(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for idx in range(1, self.hidden_layer_num+2):
            grads['W' + str(idx)] = self.layers['Affine' + str(idx)].dW + self.weight_decay_lambda * self.layers['Affine' + str(idx)].W
            grads['b' + str(idx)] = self.layers['Affine' + str(idx)].db

        return grads















(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

# 오버피팅을 재현하기 위해 학습 데이터 수를 줄임
x_train = x_train[:300]
t_train = t_train[:300]

# weight decay（가중치 감쇠） 설정 =======================
#weight_decay_lambda = 0 # weight decay를 사용하지 않을 경우
weight_decay_lambda = 0.1
# ====================================================

network = MultiLayerNet(input_size=784, hidden_size_list=[100, 100, 100, 100, 100, 100], output_size=10,
                        weight_decay_lambda=weight_decay_lambda)
optimizer = SGD(lr=0.01) # 학습률이 0.01인 SGD로 매개변수 갱신

max_epochs = 201
train_size = x_train.shape[0]
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
epoch_cnt = 0

for i in range(1000000000):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grads = network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print("epoch:" + str(epoch_cnt) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc))

        epoch_cnt += 1
        if epoch_cnt >= max_epochs:
            break


# 그래프 그리기==========
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, train_acc_list, marker='o', label='train', markevery=10)
plt.plot(x, test_acc_list, marker='s', label='test', markevery=10)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()


### 드롭아웃

In [ ]:
class Dropout:
    def __init__(self, dropout_ratio=0.5):
     self.dropout_ration = dropout_ratio
     self.mask = None

    def forward(self, x, train_flg = True):
      if train_flg:
        self.mask = np.random.rand(*x.shape) > self.dropout_ratio
        return x*self.mask
      else:
        return x * (1.0 - self.dropout_ratio)
        
    def backward(self, dout):
      return dout * self.mask


In [ ]:
# coding: utf-8
import sys, os
sys.path.append("/content/drive/MyDrive/bottom/deep-learning-from-scratch-master/deep-learning-from-scratch-master/ch03")  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.optimizer import *

class Trainer:
    """신경망 훈련을 대신 해주는 클래스
    """
    def __init__(self, network, x_train, t_train, x_test, t_test,
                 epochs=20, mini_batch_size=100,
                 optimizer='SGD', optimizer_param={'lr':0.01}, 
                 evaluate_sample_num_per_epoch=None, verbose=True):
        self.network = network
        self.verbose = verbose
        self.x_train = x_train
        self.t_train = t_train
        self.x_test = x_test
        self.t_test = t_test
        self.epochs = epochs
        self.batch_size = mini_batch_size
        self.evaluate_sample_num_per_epoch = evaluate_sample_num_per_epoch

        # optimzer
        optimizer_class_dict = {'sgd':SGD, 'momentum':Momentum, 'nesterov':Nesterov,
                                'adagrad':AdaGrad, 'rmsprpo':RMSprop, 'adam':Adam}
        self.optimizer = optimizer_class_dict[optimizer.lower()](**optimizer_param)
        
        self.train_size = x_train.shape[0]
        self.iter_per_epoch = max(self.train_size / mini_batch_size, 1)
        self.max_iter = int(epochs * self.iter_per_epoch)
        self.current_iter = 0
        self.current_epoch = 0
        
        self.train_loss_list = []
        self.train_acc_list = []
        self.test_acc_list = []

    def train_step(self):
        batch_mask = np.random.choice(self.train_size, self.batch_size)
        x_batch = self.x_train[batch_mask]
        t_batch = self.t_train[batch_mask]
        
        grads = self.network.gradient(x_batch, t_batch)
        self.optimizer.update(self.network.params, grads)
        
        loss = self.network.loss(x_batch, t_batch)
        self.train_loss_list.append(loss)
        if self.verbose: print("train loss:" + str(loss))
        
        if self.current_iter % self.iter_per_epoch == 0:
            self.current_epoch += 1
            
            x_train_sample, t_train_sample = self.x_train, self.t_train
            x_test_sample, t_test_sample = self.x_test, self.t_test
            if not self.evaluate_sample_num_per_epoch is None:
                t = self.evaluate_sample_num_per_epoch
                x_train_sample, t_train_sample = self.x_train[:t], self.t_train[:t]
                x_test_sample, t_test_sample = self.x_test[:t], self.t_test[:t]
                
            train_acc = self.network.accuracy(x_train_sample, t_train_sample)
            test_acc = self.network.accuracy(x_test_sample, t_test_sample)
            self.train_acc_list.append(train_acc)
            self.test_acc_list.append(test_acc)

            if self.verbose: print("=== epoch:" + str(self.current_epoch) + ", train acc:" + str(train_acc) + ", test acc:" + str(test_acc) + " ===")
        self.current_iter += 1

    def train(self):
        for i in range(self.max_iter):
            self.train_step()

        test_acc = self.network.accuracy(self.x_test, self.t_test)

        if self.verbose:
            print("=============== Final Test Accuracy ===============")
            print("test acc:" + str(test_acc))



### 적절한 하이퍼파라미터 값 찾기

In [ ]:
# coding: utf-8
import numpy as np


def smooth_curve(x):
    """손실 함수의 그래프를 매끄럽게 하기 위해 사용
    
    참고：http://glowingpython.blogspot.jp/2012/02/convolution-with-numpy.html
    """
    window_len = 11
    s = np.r_[x[window_len-1:0:-1], x, x[-1:-window_len:-1]]
    w = np.kaiser(window_len, 2)
    y = np.convolve(w/w.sum(), s, mode='valid')
    return y[5:len(y)-5]


def shuffle_dataset(x, t):
    """데이터셋을 뒤섞는다.

    Parameters
    ----------
    x : 훈련 데이터
    t : 정답 레이블
    
    Returns
    -------
    x, t : 뒤섞은 훈련 데이터와 정답 레이블
    """
    permutation = np.random.permutation(x.shape[0])
    x = x[permutation,:] if x.ndim == 2 else x[permutation,:,:,:]
    t = t[permutation]

    return x, t

def conv_output_size(input_size, filter_size, stride=1, pad=0):
    return (input_size + 2*pad - filter_size) / stride + 1


def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    
    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    
    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩
    
    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]
